In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:


# 2. Charger les données
# Charger les données prétraitées
df = pd.read_csv("../preprocessing/preprocessed_timeseries.csv", parse_dates=True, index_col=0)

# S'assurer que l'index est une colonne de type datetime
df.index = pd.to_datetime(df.index)

# Identifier les colonnes cibles (toutes sauf la date)
target_columns = df.columns.tolist()

# Dictionnaire pour stocker les prédictions
forecast_results = {}

# 3. Normalisation des données
# LSTM fonctionne mieux avec des données normalisées entre 0 et 1 :
scalers = {}
scaled_data = {}

for column in target_columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data[column] = scaler.fit_transform(df[[column]])
    scalers[column] = scaler

# 4. Transformer les données en séquences pour LSTM
def create_sequences(data, seq_length=12):
    """
    Transforme les données en séquences pour le modèle LSTM.
    Chaque séquence contient seq_length valeurs passées pour prédire la suivante.
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

#seq_length: Nombre de pas de temps utilisés pour la prédiction
detected_freq = pd.infer_freq(df.index)
print("***",detected_freq,"***")
if detected_freq in ["D", "B"]:  # Quotidienne ou Business Days
    seq_length = 10
elif detected_freq in ["MS", "M"]:# "justify your choices"
    seq_length = 12
else:
    seq_length = 1
print(seq_length)
# Dictionnaire pour stocker les séquences
sequences = {}

for column in target_columns:
    X, y = create_sequences(scaled_data[column], seq_length)
    sequences[column] = (X, y)

# 5. Séparer les données en Train (80%) et Test (20%)
train_size = int(0.8 * len(df))

train_sequences = {}
test_sequences = {}

for column in target_columns:
    X, y = sequences[column]
    train_sequences[column] = (X[:train_size], y[:train_size])
    test_sequences[column] = (X[train_size:], y[train_size:])

# 6. Définir l'architecture du modèle LSTM
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(200, activation='relu', return_sequences=True, input_shape=input_shape),
        LSTM(200, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# 7. Entraîner un modèle LSTM par colonne
for column in target_columns:
    print(f"Training LSTM model for {column}...")

    X_train, y_train = train_sequences[column]
    X_test, y_test = test_sequences[column]

    # Construire et entraîner le modèle
    model = build_lstm_model((seq_length, 1))
    model.fit(X_train, y_train, epochs=200, batch_size=16, verbose=1)

    # Faire des prédictions
    y_pred = model.predict(X_test)

    # Inverser la normalisation
    y_pred_rescaled = scalers[column].inverse_transform(y_pred)
    y_test_rescaled = scalers[column].inverse_transform(y_test)

    # Calculer l'erreur
    mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
    print(f"MSE for {column}: {mse:.4f}")

    # Sauvegarder les résultats
    forecast_results[column] = pd.DataFrame({
        "ds": df.index[train_size + seq_length:],
        "yhat": y_pred_rescaled.flatten()
    })

    # Visualisation
    plt.figure(figsize=(10, 5))
    plt.plot(df.index[:train_size], df[column][:train_size], label="Train")
    plt.plot(df.index[train_size:], df[column][train_size:], label="Test", color="orange")
    plt.plot(df.index[train_size + seq_length:], y_pred_rescaled, label="Forecast", linestyle="dashed", color="red")
    plt.title(f"Prédictions LSTM pour {column}")
    plt.legend()
    plt.show()

# Sauvegarder les prévisions
for col, forecast in forecast_results.items():
    forecast.to_csv(f"forecast_lstm_{col}.csv", index=False)

print("Toutes les prédictions LSTM ont été enregistrées.")